import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from glob import glob
import librosa
import librosa.display
import IPython.display as ipd
import pyAudioAnalysis
from itertools import cycle
from pyo import *
import soundfile as sf
from pydub import AudioSegment
import io


Installing Dependencies

In [ ]:
%pip install matplotlib
%pip install seaborn
%pip install librosa
%pip install IPython
%pip install pyAudioAnalysis
%pip install pydub
%pip install pyo

Note: you may need to restart the kernel to use updated packages.


Loading Audio data

In [54]:
audio=ipd.Audio('trial1.mp3')

In [ ]:
y, sr=librosa.load('C:\\Users\\Leena\\Downloads\\trial1.mp3')
print(y)

Plotting audio data

In [ ]:
pd.Series(y).plot(figsize=(10, 5),lw=1,title="Raw Audio Samples")

Trimmed audio data

In [ ]:
y_trimmed=librosa.effects.trim(y)
pd.Series(y).plot(figsize=(10, 5),lw=1,title="Trimmed Audio Samples")


In [ ]:
d=librosa.stft(y)
s_db=librosa.amplitude_to_db(np.abs(d),ref=np.max)
s_db.shape

In [ ]:
fig, ax= plt.subplots(figsize=(10,5))
img=librosa.display.specshow(s_db,x_axis="time",y_axis="log",ax=ax)
ax.set_title("Spectogram Example", fontsize=20)
fig.colorbar(img,ax=ax,format=f'%0.2f')

MELS SPECTRUM BELOW 

In [ ]:
S = librosa.feature.melspectrogram(y=y,sr=sr,n_mels=128 * 2,)
S_db_mel = librosa.amplitude_to_db(S, ref=np.max)



fig, ax = plt.subplots(figsize=(10, 5))
img = librosa.display.specshow(S_db_mel,
                              x_axis='time',
                              y_axis='log',
                              ax=ax)
ax.set_title('Mel Spectogram Example', fontsize=20)
fig.colorbar(img, ax=ax, format=f'%0.2f')
plt.show()

Feature Extraction:

Jitter 

In [ ]:
filename="C:\\Users\\Leena\\Downloads\\trial1.mp3"
y, sr = librosa.load(filename, sr=None)

# Extract pitch (fundamental frequency) using the SchmittTrigger algorithm
f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

# Calculate jitter from the pitch values
jitter = np.mean(np.abs(np.diff(f0)))

# Print or use jitter as needed
print("Jitter value:", jitter)

Shimmer

In [66]:
y, sr = librosa.load(filename, sr=None)

# Calculate the amplitude envelope
amplitude_envelope = np.abs(librosa.effects.preemphasis(y))

# Calculate shimmer from the amplitude envelope
shimmer = np.mean(np.abs(np.diff(amplitude_envelope)))

# Print or use shimmer as needed
print("Shimmer value:", shimmer)

Shimmer value: 0.0017069288


Pitch Period Entropy (PPE)

In [ ]:


audio_path = 'C:\\Users\\Leena\\Downloads\\trial1.mp3'
y, sr = librosa.load(audio_path)

pitch, magnitudes = librosa.core.piptrack(y=y, sr=sr)

# Calculate pitch period entropy
pitch_histogram, _ = np.histogram(pitch.flatten(), bins='auto', density=True)
pitch_period_entropy = -np.sum(pitch_histogram * np.log2(pitch_histogram + 1e-10))

plt.figure(figsize=(12, 8))
librosa.display.specshow(librosa.amplitude_to_db(magnitudes, ref=np.max), y_axis='log', x_axis='time')
plt.colorbar()
plt.title('Spectrogram with Pitch Periods')
plt.show()

print(f'Pitch Period Entropy: {pitch_period_entropy}')


Spread2 Value

In [ ]:

def calculate_spread2(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    pitches, magnitudes = librosa.core.piptrack(y=y)
    pitch_contour = []
    for frame in pitches.T:
        nonzero_values = frame[frame > 0]
        if len(nonzero_values) > 0:
            pitch_contour.append(np.mean(nonzero_values))
        else:
            pitch_contour.append(0)

    # Calculate Spread2
    spread2 = np.std(np.diff(pitch_contour))

    return spread2

audio_file_path = 'C:\\Users\\Leena\\Downloads\\trial1.mp3'
spread2_value = calculate_spread2(audio_file_path)
print("Spread2 Value:", spread2_value)

MDVP:Shimmer(dB)

In [65]:

def calculate_shimmer(audio_file):
    y, sr = librosa.load(audio_file, sr=None, mono=True)

    # Calculate pitch using librosa
    pitches, magnitudes = librosa.core.piptrack(y=y)

    # Extract pitch contour
    pitch_contour = []
    for frame in pitches.T:
        nonzero_values = frame[frame > 0]
        if len(nonzero_values) > 0:
            pitch_contour.append(np.mean(nonzero_values))
        else:
            pitch_contour.append(0)

    # Calculate Shimmer
    shimmer = np.mean(np.abs(np.diff(pitch_contour)))

    # Normalize shimmer values
    normalized_shimmer = shimmer / np.max(pitch_contour)

    print("MDVP:Shimmer(dB):", normalized_shimmer)


calculate_shimmer('C:\\Users\\Leena\\Downloads\\trial1.mp3')


MDVP:Shimmer(dB): 0.03591463183396145


MDVP:APQ

In [64]:

def calculate_mdvp_apq(audio_file_path):

    y, sr = librosa.load(audio_file_path)

    amplitude_feature = extract_amplitude_feature(y)
    perturbation_feature = extract_perturbation_feature(y)

    mdvp_apq = amplitude_feature / perturbation_feature

    return mdvp_apq

def extract_amplitude_feature(audio_signal):
    amplitude_feature = np.mean(np.abs(audio_signal))
    return amplitude_feature

def extract_perturbation_feature(audio_signal):
    
    perturbation_feature = np.std(audio_signal)
    return perturbation_feature


audio_file_path = "C:\\Users\\Leena\\Downloads\\trial1.mp3"
mdvp_apq_value = calculate_mdvp_apq(audio_file_path)
print(f"MDVP:APQ Value: {mdvp_apq_value}")


MDVP:APQ Value: 0.6477829217910767


MDVP:Shimmer

In [67]:


def calculate_mdvp_shimmer(audio_file_path):
    # Load the audio file
    y, sr = librosa.load(audio_file_path)

    # Extract relevant features using LibROSA or other libraries
    # Here, you may need to replace these placeholders with the actual feature extraction code
    amplitude_variation = extract_amplitude_variation(y)

    # Calculate MDVP:Shimmer based on the extracted features
    mdvp_shimmer = amplitude_variation / np.mean(np.abs(y))

    return mdvp_shimmer

def extract_amplitude_variation(audio_signal):
    # Replace this placeholder with the actual code to extract amplitude variation feature
    amplitude_variation = np.mean(np.abs(np.diff(audio_signal)))
    return amplitude_variation

# Example usage
audio_file_path = "C:\\Users\\Leena\\Downloads\\trial1.mp3"
mdvp_shimmer_value = calculate_mdvp_shimmer(audio_file_path)
print(f"MDVP:Shimmer Value: {mdvp_shimmer_value}")


MDVP:Shimmer Value: 0.20039930939674377


In [ ]:
MDVP:F0

In [68]:
def calculate_average_f0(audio_file_path):
    y, sr = librosa.load(audio_file_path)
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

    # Calculate the average F0
    average_f0 = np.mean(f0[voiced_flag])

    return average_f0

audio_file_path = "C:\\Users\\Leena\\Downloads\\trial1.mp3"
average_f0_value = calculate_average_f0(audio_file_path)
print(f"Average F0: {average_f0_value} Hz")


Average F0: 118.49143785323072 Hz


MDVP:FHI

In [69]:

def calculate_max_f0(audio_file_path):

    y, sr = librosa.load(audio_file_path)

    # Extract the fundamental frequency (F0) using LibROSA
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

    # Filter out unvoiced segments and calculate the maximum F0
    max_f0 = np.max(f0[voiced_flag])

    return max_f0

audio_file_path = "C:\\Users\\Leena\\Downloads\\trial1.mp3"
max_f0_value = calculate_max_f0(audio_file_path)
print(f"Maximum F0: {max_f0_value} Hz")


Maximum F0: 206.4563592676515 Hz


Shimmer:DDA

In [ ]:

import librosa
import numpy as np

def calculate_shimmer_dda(audio_file_path):
    # Load the audio file
    y, sr = librosa.load(audio_file_path)

    # Extract relevant features using LibROSA or other libraries
    # Here, you may need to replace these placeholders with the actual feature extraction code
    amplitude_variation = extract_amplitude_variation(y)

    # Calculate Shimmer:DDA based on the extracted features
    shimmer_dda = amplitude_variation / np.mean(np.abs(y))

    return shimmer_dda

def extract_amplitude_variation(audio_signal):
    # Replace this placeholder with the actual code to extract amplitude variation feature
    amplitude_variation = np.mean(np.abs(np.diff(audio_signal)))
    return amplitude_variation

# Example usage
audio_file_path = "C:\\Users\\Leena\\Downloads\\trial1.mp3"
shimmer_dda_value = calculate_shimmer_dda(audio_file_path)
print(f"Shimmer:DDA Value: {shimmer_dda_value}")
